In [97]:
import pandas as pd
import numpy as np
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics.pairwise import cosine_similarity

In [98]:
file = 'GLCM_local_weights.csv'
csv_weights = np.genfromtxt(file, delimiter=',')

# csv_weights = pd.read_csv('GLCM_local_weights.csv',header=0) #Change the filename for different weights

In [99]:
weight_data = csv_weights

In [100]:
weight_data

array([[ 7.09963496e-02, -6.52207114e-01, -1.69463561e-01,
        -5.78595060e-02,  6.01185264e-02, -3.26233892e-02,
         1.29912014e-01, -1.21947164e-01, -2.52868866e-01,
        -2.52496839e-01,  1.11810951e-01, -4.11361983e-03,
        -4.82965551e-01, -2.75579514e-01,  3.15260689e-01],
       [-1.60531249e-01,  6.92871674e-01, -1.52004382e-01,
         2.24123832e-01, -2.02939128e-01,  1.31488593e-01,
        -1.03078678e-01,  1.67368997e-01,  6.75959285e-01,
         6.79748522e-01, -1.05523899e+00, -5.48813055e-01,
        -1.15994443e+00,  7.83665835e-01, -2.24271700e-01],
       [ 9.91693078e-01, -1.54840643e-01, -1.25103997e+01,
        -1.95012557e+00,  1.37814142e+01,  7.64497338e+00,
         9.66744073e-01, -2.41758689e+00, -1.05250176e+01,
        -2.43071978e+00, -1.26924407e+00, -1.13502968e+00,
        -7.39387354e-01, -2.20706537e-01,  1.51362099e+00],
       [ 1.38963669e+00,  1.48739361e-01, -9.16119091e+00,
        -1.98378804e+00,  1.04444459e+01,  4.60017006

In [101]:
cols = [
    'GLCM_Energy_1'
    ,'GLCM_corr_1'
    ,'GLCM_diss_1'
    ,'GLCM_hom_1'
    ,'GLCM_contr_1'
    ,'GLCM_Energy_2'
    ,'GLCM_corr_2'
    ,'GLCM_diss_2'
    ,'GLCM_hom_2'
    ,'GLCM_contr_2'
    ,'GLCM_Energy_3'
    ,'GLCM_corr_3'
    ,'GLCM_diss_3'
    ,'GLCM_hom_3'
    ,'GLCM_contr_3'
    ,'Y'
    # ,'saliency_matrix'
]
excel_data = pd.read_excel('glcm_with_clusters.xlsx', header=0, usecols=range(100))
#data = excel[cols].sample(frac=1).values
data = excel_data[cols].values

# X is all features, y is classified value
X = data[:, 0:-1]
y = data[:, -1]

no_of_instances = X.shape[0]
no_of_features = X.shape[1]

In [107]:
diff_fns = {
    'GLCM_Energy_1'   : lambda x, y: abs(x-y)
    ,'GLCM_corr_1'    : lambda x, y: abs(x-y)
    ,'GLCM_diss_1'    : lambda x, y: abs(x-y)
    ,'GLCM_hom_1'     : lambda x, y: abs(x-y)
    ,'GLCM_contr_1'   : lambda x, y: abs(x-y)
    ,'GLCM_Energy_2'  : lambda x, y: abs(x-y)
    ,'GLCM_corr_2'    : lambda x, y: abs(x-y)
    ,'GLCM_diss_2'    : lambda x, y: abs(x-y)
    ,'GLCM_hom_2'     : lambda x, y: abs(x-y)
    ,'GLCM_contr_2'   : lambda x, y: abs(x-y)
    ,'GLCM_Energy_3'  : lambda x, y: abs(x-y)
    ,'GLCM_corr_3'    : lambda x, y: abs(x-y)
    ,'GLCM_diss_3'    : lambda x, y: abs(x-y)
    ,'GLCM_hom_3'     : lambda x, y: abs(x-y)
    ,'GLCM_contr_3'   : lambda x, y: abs(x-y)
    }

sim_fns = {
    'GLCM_Energy_1'   : lambda x, y: abs(min(x,y)/max(x,y))
    ,'GLCM_corr_1'    : lambda x, y: abs(min(x,y)/max(x,y))
    ,'GLCM_diss_1'    : lambda x, y: abs(min(x,y)/max(x,y))
    ,'GLCM_hom_1'     : lambda x, y: abs(min(x,y)/max(x,y))
    ,'GLCM_contr_1'   : lambda x, y: abs(min(x,y)/max(x,y))
    ,'GLCM_Energy_2'  : lambda x, y: abs(min(x,y)/max(x,y))
    ,'GLCM_corr_2'    : lambda x, y: abs(min(x,y)/max(x,y))
    ,'GLCM_diss_2'    : lambda x, y: abs(min(x,y)/max(x,y))
    ,'GLCM_hom_2'     : lambda x, y: abs(min(x,y)/max(x,y))
    ,'GLCM_contr_2'   : lambda x, y: abs(min(x,y)/max(x,y))
    ,'GLCM_Energy_3'  : lambda x, y: abs(min(x,y)/max(x,y))
    ,'GLCM_corr_3'    : lambda x, y: abs(min(x,y)/max(x,y))
    ,'GLCM_diss_3'    : lambda x, y: abs(min(x,y)/max(x,y))
    ,'GLCM_hom_3'     : lambda x, y: abs(min(x,y)/max(x,y))
    ,'GLCM_contr_3'   : lambda x, y: abs(min(x,y)/max(x,y))
    }

In [108]:
weight_data.shape

(48, 15)

In [109]:
def predict( train_index, test_index):
    sim_val_arr = []

    for ti_idx, ti in enumerate(train_index):
        sim_val = 0
        
        for idx, Xi in enumerate(X[ti]):
            Ti = X[test_index[0]][idx]
            
            attrib = cols[idx]
                        
            # Similarity value from the sim functions multiplied by the weight
            weight_data[ti_idx][idx]
#             print(sim_fns[attrib](Xi,Ti), weight_data[ti_idx][idx])
            sim_val += sim_fns[attrib](Xi,Ti) * weight_data[ti_idx][idx]
            break
        # For each instance append similarity score
        sim_val_arr.append(sim_val)
    
    sim_val_arr = np.array(sim_val_arr)

    nearest_idx = sim_val_arr.argsort()[-1:][::-1]

    k_nearest_solutions = [y[l] for l in nearest_idx]
    #print(k_nearest_match_cases, k_nearest_solutions)
    return k_nearest_solutions

In [110]:
for train_index, test_index in LeaveOneOut().split(X):
    print(train_index, test_index)
    
    predicted = predict(train_index,test_index)
    actual = y[test_index[0]]
    
    print(predicted, actual)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] [0]
[0.0] 0.0
[ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] [1]
[1.0] 0.0
[ 0  1  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] [2]
[0.0] 0.0
[ 0  1  2  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] [3]
[1.0] 0.0
[ 0  1  2  3  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] [4]
[1.0] 0.0
[ 0  1  2  3  4  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] [5]
[1.0] 0.0
[ 0  1  2  3  4  5  7  8  9 10 11 12 13 14 15 16 17 

In [112]:
k=1
def test_accuracy():
    correct_total = 0

    # Leave one out cross validation
    for train_index, test_index in LeaveOneOut().split(X):
        predicted = predict(train_index,test_index)
        actual = y[test_index[0]]
        is_correct = 1 if actual in predicted else 0
        print(actual,predicted, is_correct)
        correct_total+=is_correct

    accuracy = correct_total / y.shape[0]
    print("Accuracy for {} neighbors is {}%: ".format(k,accuracy*100))

In [113]:
test_accuracy()

0.0 [0.0] 1
0.0 [1.0] 0
0.0 [0.0] 1
0.0 [1.0] 0
0.0 [1.0] 0
0.0 [1.0] 0
0.0 [1.0] 0
0.0 [1.0] 0
0.0 [0.0] 1
0.0 [1.0] 0
0.0 [1.0] 0
0.0 [1.0] 0
0.0 [1.0] 0
0.0 [0.0] 1
0.0 [0.0] 1
0.0 [0.0] 1
0.0 [1.0] 0
0.0 [1.0] 0
0.0 [0.0] 1
0.0 [1.0] 0
0.0 [1.0] 0
0.0 [0.0] 1
0.0 [0.0] 1
0.0 [0.0] 1
0.0 [0.0] 1
0.0 [0.0] 1
1.0 [0.0] 0
1.0 [0.0] 0
1.0 [0.0] 0
1.0 [0.0] 0
1.0 [1.0] 1
1.0 [1.0] 1
1.0 [1.0] 1
1.0 [0.0] 0
1.0 [1.0] 1
1.0 [0.0] 0
1.0 [1.0] 1
1.0 [0.0] 0
1.0 [0.0] 0
1.0 [1.0] 1
1.0 [1.0] 1
1.0 [1.0] 1
1.0 [1.0] 1
1.0 [1.0] 1
1.0 [1.0] 1
1.0 [1.0] 1
1.0 [1.0] 1
1.0 [1.0] 1
Accuracy for 1 neighbors is 54.166666666666664%: 
